A **graph-based organism** that (1) **grows** from a single node via a neural cellular automaton acting on a graph, then (2) **moves** by contracting edges through local signals only.

---

# Experiment: NCA-grown graph creature with edge-contraction locomotion

## 0) World & physics

* **Space:** 2D plane with gravity and ground contact.
* **Body:** an undirected **graph** $G=(V,E)$.
  Each node $i\in V$ has a position $x_i\in\mathbb{R}^2$, mass $m$, and state vector $h_i\in\mathbb{R}^D$.
  Each edge $e=(i,j)\in E$ is a **contractile spring** with rest length $L_{ij}$ and stiffness $k_{ij}$.
* **Actuation:** a node can send a scalar **signal** $u_{i\to j}\in[0,1]$ to each adjacent edge; the edge’s **effective rest length** becomes

  $$
  \tilde{L}_{ij} = L_{ij}\big(1 - \alpha\cdot \text{sat}(u_{i\to j} + u_{j\to i})\big)
  $$

  with contraction coefficient $\alpha\in(0,1)$ and $\text{sat}(\cdot)$ clipping to $[0,1]$.
* **Forces:** spring forces, damping, gravity; Coulomb friction with the ground; self-collision avoidance (repulsive penalty at short distances).
* **Constraints:** degree limit per node (e.g., ≤4), min edge length ≥ $L_\text{min}$, planarity (optional).

---

## 1) Two-phase NCA architecture (local rules only)

You’ll use *two* tiny NCAs (message-passing GNNs with shared weights across nodes):

### A) **Developmental NCA** $f_\theta^{\text{grow}}$ (time $t=1..T_g$)

* **Inputs per node:** its state $h_i$, degree, local geometric features (edge lengths/angles), and **diffusing morphogens** (a few scalar channels aggregated from neighbors).
* **Message passing:** for each edge $i\!-\!j$, compute a message $m_{i\leftarrow j}$ from $[h_j,\ \text{geom}_{ij}]$. Aggregate by sum/mean.
* **Updates:**

  * **State update:** $h_i \leftarrow \text{GRU}(h_i,\ \text{agg}(m_{i\leftarrow *}))$
  * **Growth logits:** node proposes actions with local probabilities:

    * **BUD**: create a new node $k$ at $x_k = x_i + \delta$ (unit step in a proposed direction) and connect $i\!-\!k$.
    * **CONNECT**: connect to a nearby non-neighbor $j$ if within radius $r_c$ and planarity/degree allow.
    * **PRUNE**: drop weakest edge if degree > threshold or local stress low.
    * **TYPE**: set local **tissue type** bit(s): passive / muscle-preferred / anchor (higher friction).
  * Enforce budgets: total nodes ≤ $N_\text{max}$, edges ≤ $E_\text{max}$; stochastic sampling with temperature to keep it local.
* **Regularizers:** penalty for skinny tendrils (high average edge slenderness), disconnected components, and over-budget growth.

### B) **Control NCA** $f_\phi^{\text{act}}$ (time $t=T_g+1..T_g+T_c$)

* **Inputs per node each control tick:** its hidden state $h_i$ (dev carries over), neighbor summaries (mean of neighbor $h_j$, edge lengths vs rest), ground contact flag, and **one or two oscillatory pacer channels** that diffuse (for emergent CPGs).
* **Outputs:**

  * per-edge contraction commands $u_{i\to j}$,
  * optional **dock/undock** bit (increases friction when “foot” presses),
  * small bias to **preferred heading** vector in $h_i$ for coordination.
* **Asynchrony:** update a random subset of nodes each tick (simulates network jitters).

> One set of weights $\{\theta,\phi\}$ is shared by all nodes; **no global coordinator**.

---

## 2) Training objective & curriculum

### Primary task

* Start from **one seed node** on flat ground.
* **Grow** for $T_g$ steps into a morphology.
* **Move** for $T_c$ seconds via edge contraction signals to **maximize forward displacement** along +x.

### Fitness (scalarized or Pareto)

1. **Displacement** $D_x$ of the body centroid by $T_g+T_c$ (maximize).
2. **Energy cost** $E_u = \sum_{t,e} |u_e(t)|$ (minimize).
3. **Stability** penalty: sum of slip ratio & loss of ground contact for all “feet”.
4. **Material cost**: $N$ nodes + $E$ edges (favor minimal bodies).
5. **Integrity**: connectivity maintained; self-collision violations penalized.

### Curriculum (domain randomization each episode)

* Phase 1: flat ground, low friction variation, shorter body budget.
* Phase 2: add **terrain bumps/holes**, friction patches, small slopes (±5°).
* Phase 3: **damage** at $t=T_g + T_c/2$: remove 10–20% nodes in a local region; measure **recovery displacement** afterward.
* Phase 4 (generalization tests): unseen slopes, heavier damping, limited contraction $\alpha$.

---

## 3) Optimization strategy

Two workable routes:

### Route A — **Evolutionary** (robust, simple to implement)

* **Genome:** weights of $f_\theta^{\text{grow}}$ and $f_\phi^{\text{act}}$ + a few hyper-params (bud radius, budgets, contraction α).
* **Algorithm:** **MAP-Elites** (quality-diversity) with behavior descriptors: body length, limb count, duty cycle symmetry, ground contact ratio; then optionally refine elites via CMA-ES.
* **Why:** gradients through growth + contact physics are messy; evolution handles non-differentiabilities.

### Route B — **Differentiable co-design** (if you have differentiable physics)

* Train with backprop using a differentiable mass-spring/contact engine.
* Alternate updates: a few steps for $\theta$ (growth) holding $\phi$ fixed, then for $\phi$ (control) holding $\theta$ fixed; add **straight-through estimators** for discrete growth actions.

---

## 4) Key constraints that make it work (and safe)

* **Locality only:** nodes use only 1-hop info + aggregated messages. No global coordinates.
* **Bud validity checks:** before adding an edge/node, check degree ≤ 4, min angle between edges ≥ 30°, no edge crossings (planarity test within 2 hops).
* **Physics sanity:** clamp spring stiffness $k$, contraction $\alpha \le 0.3$, damping high enough to avoid chatter.
* **Friction modulation = feet:** allow “anchor type” nodes to temporarily raise ground friction when normal force is high → enables **gait phases** without global timing.

---

## 5) Metrics & analysis

* **Locomotion score**: $D_x/T_c$ (m/s).
* **Specific cost of transport**: $E_u / (m g D_x)$.
* **Stability**: variance of center-of-pressure; % time with ≥2 contact points.
* **Morphology stats**: limb count (graph leafs), average limb length, branching factor, symmetry index.
* **CPG emergence**: spectral peak of edge-contraction patterns per limb.
* **Robustness curves**: displacement vs node-removal fraction; vs friction noise.

---

## 6) Baselines & ablations

* **Fixed body + learned controller** (no growth).
* **Hand-grown symmetric body + learned controller**.
* **Single NCA doing both growth+control** vs **two separate NCAs** (your design).
* **No anchor friction modulation** ablation.
* **Synchronous vs asynchronous updates**.
* **No morphogen diffusion** ablation (does symmetry still emerge?).

---

## 7) Default hyper-parameters (good starting point)

* **Graph limits:** $N_\text{max}=60$ nodes, $E_\text{max}=120$.
* **Spring constants:** $k=150\,\mathrm{N/m}$, damping $c=3\,\mathrm{Ns/m}$, $L_\text{min}=0.04\,\mathrm{m}$, initial $L_{ij}=0.06\,\mathrm{m}$, contraction $\alpha=0.25$.
* **Ground:** friction μ=0.8 (randomize ±0.2), restitution 0.0.
* **NCAs:** hidden/state dim $D=24$. Messages 32-64 dims. Two MP layers with GRU update per tick.
* **Timings:** $T_g=150$ growth ticks (dt=0.03 s), $T_c=600$ control ticks.
* **Evolution:** POP=256, MAP-Elites grid on (limb count × duty symmetry), 500 generations, 6 randomized evals per genome.

---

## 8) Minimal per-tick logic (conceptual)

**Growth tick (per node $i$)**

```
msgs = { MLP_msg([h_j, geom(i,j)]) for j in N(i) }
agg = sum(msgs)
h_i = GRU(h_i, agg)
[p_bud, p_connect, p_prune, type_logits] = Head(h_i)
if sample(p_bud) and budget_ok and planarity_ok:
    add_node_and_edge(i)
if sample(p_connect) and neighbor_within_rc and planarity_ok:
    add_edge(i,j)
if sample(p_prune) and degree(i)>1:
    drop_weakest_edge(i)
type_i = argmax(type_logits)  # passive/muscle/anchor
```

**Control tick (per node $i$)**

```
msgs = { MLP_msg_ctrl([h_j, edge_state(i,j), contact_j]) for j in N(i) }
agg = sum(msgs)
h_i = GRU_ctrl(h_i, agg)
for j in N(i):
    u[i->j] = sigmoid(Head_u([h_i, edge_state(i,j)]))
anchor[i] = sigmoid(Head_anchor(h_i)) > τ
```

Physics integrates forces with $\tilde{L}_{ij}(t)$, friction depends on `anchor`.

---

## 9) Failure modes & fixes

* **Jelly wobble (no net motion):** raise damping, add energy penalty; introduce anchor friction modulation to create stance phases.
* **Spaghetti growth:** increase planarity/angle constraints, add tendon-length regularizer.
* **Collapse under actuation:** cap $\alpha$, add **tension penalty** when edges exceed safe strain.
* **Overfitting to terrain:** domain randomize friction, bumps, slope.
* **No CPG emerges:** add a **slowly diffusing pacer channel** seeded at root; train with mild activation noise.

---

## 10) What makes this publishable

* **Developmental control → locomotion**: growth *and* movement from the same local paradigm, no global planner.
* **Graph NCA (not grid)** on continuous physics with edge-actuation only.
* **Robustness**: self-healing after node removal, maintained gait.
* **Interpretability**: visualize learned morphogen channels and show they function as CPGs and role markers (anchor vs muscle).

---

**starter code scaffold** (Python) with:

* a mass-spring physics core,
* the graph NCA modules (growth + control) using message passing,
* MAP-Elites training loop, logging, and simple viewers.


In [16]:
import re

def parse_scientific(s):
    match = re.search(r'([\d\.]+)\s*\\cdot\s*10\^{(\d+)}', s)
    if match:
        base = float(match.group(1))
        exp = int(match.group(2))
        return base * (10 ** exp)
    return float('-inf')

lines = [
"Tetraminx & 16 & 24 & 4 & 299 & & $1.12 \cdot 10^{7}$ ",
"Little chop & 12 & 24 & 6 & & & $3.102 \cdot 10^{23}$ ",
"Dino + little chop & 28 & 24 & 6 & & & $3.102 \cdot 10^{23}$ ",
"2x2x2 & 12 & 24 & 6 & 14 & 14 & $8.818 \cdot 10^{7}$ ",
"Dino & 16 & 24 & 6 & & & $2.395 \cdot 10^{8}$ ",
"Jingpyraminx & 16 & 28 & 4 & & & $4.479 \cdot 10^{7}$ ",
"Skewb & 16 & 30 & 6 & 11 & 11 & $3.779 \cdot 10^{7}$ ",
"Skewb diamond & 16 & 32 & 8 & & & $1.659 \cdot 10^{6}$ ",
"Pyraminx & 16 & 36 & 4 & 15 & 15 & $3.023 \cdot 10^{8}$ ",
"Master pyramorphix & 6 & 36 & 4 & & & $1.327 \cdot 10^{6}$ ",
"Master morphix & 6 & 40 & 4 & & & $5.308 \cdot 10^{6}$ ",
"Helicopter & 12 & 48 & 6 & & & $1.185 \cdot 10^{19}$ ",
"Octastar & 12 & 48 & 8 & & & $3.102 \cdot 10^{23}$ ",
"2x2x2 + dino & 28 & 48 & 6 & & & $6.204 \cdot 10^{23}$ ",
"Christopher's jewel & 12 & 48 & 8 & & & $2.009 \cdot 10^{15}$ ",
"Compycube & 16 & 48 & 6 & & & $1.571 \cdot 10^{12}$ ",
"Master tetraminx & 16 & 48 & 4 & & & $2.682 \cdot 10^{15}$ ",
"2x2x2 + little chop & 24 & 48 & 6 & & & $1.925 \cdot 10^{47}$ ",
"2x2x2 + dino + little chop & 40 & 48 & 6 & & & $1.925 \cdot 10^{47}$ ",
"3x3x3 & 12 & 48 & 6 & 18 & 20 & $4.325 \cdot 10^{19}$ ",
"Trajber's octahedron & 12 & 56 & 8 & & & $4.05 \cdot 10^{19}$ ",
"Master pyraminx & 32 & 64 & 4 & 20 & 26 & $2.607 \cdot 10^{18}$ ",
"Pentultimate & 24 & 72 & 12 & & & $3.386 \cdot 10^{35}$ ",
"Curvycopter & 12 & 72 & 6 & & & $3.033 \cdot 10^{21}$ ",
"FTO & 16 & 72 & 8 & & & $1.583 \cdot 10^{29}$ ",
"Master skewb & 16 & 78 & 6 & & & $5.192 \cdot 10^{32}$ ",
"Icosamate & 24 & 80 & 20 & & & $7.113 \cdot 10^{34}$ ",
"Professor tetraminx & 32 & 88 & 4 & & & $1.846 \cdot 10^{33}$ ",
"4x4x4 & 24 & 96 & 6 & 35 & 55 & $1.697 \cdot 10^{55}$ ",
"Professor pyraminx & 32 & 100 & 4 & & & $1.495 \cdot 10^{35}$ ",
"Redicosahedron & 24 & 120 & 20 & & & $1.738 \cdot 10^{49}$ ",
"Bigchop & 30 & 120 & 12 & & & $3.462 \cdot 10^{163}$ ",
"Pyraminx crystal & 24 & 120 & 12 & 50 &  & $1.007 \cdot 10^{68}$ ",
"Megaminx & 24 & 120 & 12 & 59 & 194 & $1.007 \cdot 10^{68}$ ",
"Starminx & 40 & 120 & 12 & & & $2.483 \cdot 10^{50}$ ",
"Redicosahedron with \space \space centers & 24 & 120 & 20 & & & $1.738 \cdot 10^{49}$ ",
"Master FTO & 32 & 128 & 8 & & & $3.131 \cdot 10^{56}$ ",
"Megaminx + chopasaurus & 64 & 132 & 12 & & & $4.226 \cdot 10^{117}$ ",
"Chopasaurus & 40 & 132 & 12 & & & $4.226 \cdot 10^{117}$ ",
"Royal tetraminx & 32 & 132 & 4 & & & $2.113 \cdot 10^{57}$ ",
"Starminx combo & 64 & 132 & 12 & & & $7.095 \cdot 10^{130}$ ",
"Starminx2 & 24 & 132 & 12 & & & $7.095 \cdot 10^{130}$ ",
"Icosaminx & 24 & 140 & 20 & & & $2.115 \cdot 10^{67}$ ",
"Royal pyraminx & 48 & 144 & 4 & & & $1.712 \cdot 10^{59}$ ",
"5x5x5 & 24 & 144 & 6 & 52 & 130 & $2.583 \cdot 10^{90}$ ",
"Professor skewb & 32 & 150 & 6 & & & $4.091 \cdot 10^{74}$ ",
"Regular astrominx & 24 & 180 & 20 & & & $1.732 \cdot 10^{139}$ ",
"Emperor tetraminx & 48 & 184 & 4 & & & $3.484 \cdot 10^{83}$ ",       
"Master pentultimate & 24 & 192 & 12 & & & $1.003 \cdot 10^{158}$ ",
"Emperor pyraminx & 48 & 196 & 4 & & & $2.822 \cdot 10^{85}$ ",
]

sorted_lines = sorted(lines, key=lambda l: parse_scientific(l), reverse=False)
for line in sorted_lines:
    print(f"\t{line} ", "\\", "\\") 
    print("\t\\hline")


	Master pyramorphix & 6 & 36 & 4 & & & $1.327 \cdot 10^{6}$   \ \
	\hline
	Skewb diamond & 16 & 32 & 8 & & & $1.659 \cdot 10^{6}$   \ \
	\hline
	Master morphix & 6 & 40 & 4 & & & $5.308 \cdot 10^{6}$   \ \
	\hline
	Tetraminx & 16 & 24 & 4 & 299 & & $1.12 \cdot 10^{7}$   \ \
	\hline
	Skewb & 16 & 30 & 6 & 11 & 11 & $3.779 \cdot 10^{7}$   \ \
	\hline
	Jingpyraminx & 16 & 28 & 4 & & & $4.479 \cdot 10^{7}$   \ \
	\hline
	2x2x2 & 12 & 24 & 6 & 14 & 14 & $8.818 \cdot 10^{7}$   \ \
	\hline
	Dino & 16 & 24 & 6 & & & $2.395 \cdot 10^{8}$   \ \
	\hline
	Pyraminx & 16 & 36 & 4 & 15 & 15 & $3.023 \cdot 10^{8}$   \ \
	\hline
	Compycube & 16 & 48 & 6 & & & $1.571 \cdot 10^{12}$   \ \
	\hline
	Christopher's jewel & 12 & 48 & 8 & & & $2.009 \cdot 10^{15}$   \ \
	\hline
	Master tetraminx & 16 & 48 & 4 & & & $2.682 \cdot 10^{15}$   \ \
	\hline
	Master pyraminx & 32 & 64 & 4 & 20 & 26 & $2.607 \cdot 10^{18}$   \ \
	\hline
	Helicopter & 12 & 48 & 6 & & & $1.185 \cdot 10^{19}$   \ \
	\hline
	Trajber's octa

In [1]:
from IPython.display import display, HTML

html_code = """
<canvas id="world" width="1200" height="600" style="background:#0b0b10; display:block; margin:0 auto;"></canvas>

<div style="text-align:center; margin-top:6px; color:#ddd; font-family:Arial, sans-serif;">
  <label style="margin-right:10px">contraction amplitude
    <input id="alpha" type="range" min="0" max="0.6" step="0.01" value="0.25">
  </label>
  <label style="margin-right:10px">contraction speed
    <input id="speed" type="range" min="0.005" max="0.08" step="0.005" value="0.02">
  </label>
  <label style="margin-right:10px">contraction prob
    <input id="prob" type="range" min="0.0" max="0.2" step="0.005" value="0.02">
  </label>
  <label style="margin-right:10px">lift factor
    <input id="lift" type="range" min="0" max="1" step="0.05" value="0.9">
  </label>
  <label style="margin-right:10px">spring K
    <input id="k" type="range" min="0.02" max="0.6" step="0.01" value="0.15">
  </label>
  <button id="regen" style="margin-left:10px">regen graph</button>
</div>

<script>
(function(){
  if (window.creatureLoop) cancelAnimationFrame(window.creatureLoop);

  const canvas = document.getElementById("world");
  const ctx = canvas.getContext("2d");

  let contractionAlpha = parseFloat(document.getElementById('alpha').value);
  let contractionSpeed = parseFloat(document.getElementById('speed').value);
  let contractionProb = parseFloat(document.getElementById('prob').value);
  let liftFactor = parseFloat(document.getElementById('lift').value);
  let springK = parseFloat(document.getElementById('k').value);

  document.getElementById('alpha').oninput = e => contractionAlpha = parseFloat(e.target.value);
  document.getElementById('speed').oninput = e => contractionSpeed = parseFloat(e.target.value);
  document.getElementById('prob').oninput = e => contractionProb = parseFloat(e.target.value);
  document.getElementById('lift').oninput = e => liftFactor = parseFloat(e.target.value);
  document.getElementById('k').oninput = e => springK = parseFloat(e.target.value);

  document.getElementById('regen').onclick = () => buildConnectedGraph();

  const dt = 0.6;
  const globalDamping = 0.995;
  const baseFrictionHigh = 0.9;
  const baseFrictionLow = 0.1;

  class Node {
    constructor(x,y){
      this.x = x; this.y = y;
      this.vx = 0; this.vy = 0;
      this.radius = 6;
      this.friction = baseFrictionHigh;
    }
  }

  class Edge {
    constructor(a,b){
      this.a = a; this.b = b;
      this.rest = dist(nodes[a], nodes[b]);
      this.phase = 0;
      this.contracting = false;
    }
  }

  function dist(n1,n2){
    return Math.sqrt((n1.x-n2.x)**2 + (n1.y-n2.y)**2);
  }

  let nodes = [];
  let edges = [];
  const N_DEFAULT = 10;
  const EXTRA_EDGES = 9;

  function buildConnectedGraph(N=N_DEFAULT){
    nodes = [];
    edges = [];
    for(let i=0;i<N;i++){
      let angle = (i/N)*Math.PI*2 + (Math.random()-0.5)*0.5;
      let r = 80 + Math.random()*60;
      let cx = canvas.width/2 + Math.cos(angle)*r;
      let cy = canvas.height/2 + Math.sin(angle)*r;
      nodes.push(new Node(cx + (Math.random()-0.5)*40, cy + (Math.random()-0.5)*40));
    }
    for(let i=1;i<N;i++){
      let j = Math.floor(Math.random()*i);
      edges.push(new Edge(i,j));
    }
    let tries=0;
    while(edges.length < (N-1+EXTRA_EDGES) && tries<200){
      let a = Math.floor(Math.random()*N);
      let b = Math.floor(Math.random()*N);
      if(a!==b && !edgeExists(a,b)) edges.push(new Edge(a,b));
      tries++;
    }
  }

  function edgeExists(a,b){
    for(let e of edges) if((e.a===a&&e.b===b)||(e.a===b&&e.b===a)) return true;
    return false;
  }

  buildConnectedGraph();

  let trail = [];
  const maxTrail = 20000;

  function maybeTriggerContractions(){
    for(let e of edges){
      if(!e.contracting && Math.random()<contractionProb){
        e.contracting=true;
      }
    }
  }

  function updatePhysics(){
    maybeTriggerContractions();

    for(let e of edges){
      if(e.contracting){
        e.phase += contractionSpeed;
        if(e.phase>=1){ e.phase=1; e.contracting=false; }
      } else { e.phase -= contractionSpeed; if(e.phase<0) e.phase=0; }
    }

    for(let i=0;i<nodes.length;i++){
      let n=nodes[i];
      let maxPhase=0;
      for(let e of edges) if(e.a===i||e.b===i) if(e.phase>maxPhase) maxPhase=e.phase;
      n.friction = baseFrictionHigh*(1-liftFactor*maxPhase) + baseFrictionLow*(liftFactor*maxPhase);
    }

    for(let e of edges){
      let n1=nodes[e.a], n2=nodes[e.b];
      let dx = n2.x-n1.x, dy = n2.y-n1.y;
      let d = Math.sqrt(dx*dx+dy*dy)+1e-8;
      let L = e.rest*(1-contractionAlpha*e.phase);
      let F = springK*(d-L);
      let fx = F*dx/d, fy = F*dy/d;
      n1.vx += fx*0.5; n1.vy += fy*0.5;
      n2.vx -= fx*0.5; n2.vy -= fy*0.5;
    }

    for(let n of nodes){
      n.vx *= n.friction; n.vy *= n.friction;
      n.vx *= globalDamping; n.vy *= globalDamping;
      n.x += n.vx*dt; n.y += n.vy*dt;
    }

    let cx = nodes.reduce((s,n)=>s+n.x,0)/nodes.length;
    let cy = nodes.reduce((s,n)=>s+n.y,0)/nodes.length;
    trail.push({x:cx,y:cy});
    if(trail.length>maxTrail) trail.shift();
  }

  function draw(){
    ctx.clearRect(0,0,canvas.width,canvas.height);

    // draw trail
    ctx.strokeStyle="white";
    ctx.beginPath();
    for(let i=0;i<trail.length;i++){
      let p = trail[i];
      if(i===0) ctx.moveTo(p.x,p.y); else ctx.lineTo(p.x,p.y);
    }
    ctx.stroke();

    // draw edges
    for(let e of edges){
      let n1=nodes[e.a], n2=nodes[e.b];
      ctx.strokeStyle = e.phase>0?"red":"gray";
      ctx.lineWidth = 2;
      ctx.beginPath();
      ctx.moveTo(n1.x,n1.y);
      ctx.lineTo(n2.x,n2.y);
      ctx.stroke();
    }

    // draw nodes with friction bar and color-coded by friction
    for(let n of nodes){
      let f=(n.friction-baseFrictionLow)/(baseFrictionHigh-baseFrictionLow);
      f=Math.max(0,Math.min(1,f));
      let r = 30*(1-f)+20*f;
      let g = 200*f+80*(1-f);
      let b = 200*(1-f)+120*f;
      ctx.fillStyle=`rgb(${r},${g},${b})`;
      ctx.beginPath();
      ctx.arc(n.x,n.y,n.radius,0,Math.PI*2);
      ctx.fill();

      // friction bar
      ctx.fillStyle="#222";
      ctx.fillRect(n.x-10,n.y+10,20,4);
      ctx.fillStyle="#fff";
      ctx.fillRect(n.x-10,n.y+10,20*f,4);
    }

    // center of mass
    let cx = nodes.reduce((s,n)=>s+n.x,0)/nodes.length;
    let cy = nodes.reduce((s,n)=>s+n.y,0)/nodes.length;
    ctx.fillStyle="white";
    ctx.beginPath();
    ctx.arc(cx,cy,3,0,Math.PI*2);
    ctx.fill();
  }

  function loop(){
    updatePhysics();
    draw();
    window.creatureLoop = requestAnimationFrame(loop);
  }

  loop();
})();
</script>
"""

display(HTML(html_code))


Perfect! Let’s map out a **research roadmap** from your current simulation toward a full-fledged **graph-based NCA-controlled organism**, including experimental milestones and potential outputs.

---

## **Step 0: Current Playground – Baseline**

**Status:**

* Random connected graph “creatures” with nodes and edges.
* Physics simulation: friction, spring edges, top-down 2D environment.
* UI controls for tuning contraction amplitude, speed, probability, lift factor, spring stiffness.
* Visualizations: node color (friction), friction bars, center-of-mass trail.

**Goal:** Get a stable, adjustable platform for experimentation.

**Next step milestone:** Stabilize the simulation so that “creatures” can sustain a shape and move slowly without falling apart.

---

## **Step 1: Define Fitness Metrics**

Before adding controllers, define **what “good” behavior is**. Possible metrics:

1. **Distance traveled** – how far the center of mass moves in a given time.
2. **Stability** – how much the shape deforms while moving.
3. **Energy efficiency** – minimize sum of contraction magnitudes over time.
4. **Maneuverability / directionality** – ability to reach a target.

**Milestone:** Implement fitness computation in JS or Python to quantify performance per simulation.

---

## **Step 2: Local-rule Controllers (Graph NCA)**

**Idea:**

* Each node has a **local state vector**.
* Each edge/node interaction is **message passing**: nodes can sense neighbor positions, velocities, edge phases.
* Node outputs: contraction signals for incident edges.

**Implementation options:**

1. **Simple heuristic rule-based NCA**

   * Example: contract an edge if neighbor node is ahead, reduce friction if trailing.
   * Advantage: interpretable, easy to debug.

2. **Differentiable NCA / small neural network per node**

   * Inputs: neighbor relative positions, velocities, edge states.
   * Output: contraction magnitude for each edge.
   * Can train via **reinforcement learning** or **evolutionary algorithms**.

**Milestone:** Get a single creature to move using a **local rule**, without global control.

---

## **Step 3: Evolutionary / RL Optimization**

1. **Evolve NCA weights** or local rules using **genetic algorithms**:

   * Encode neural network weights as genome.
   * Mutate & recombine.
   * Evaluate fitness (distance moved, stability, efficiency).

2. **Reinforcement Learning approach**:

   * Treat each node as an agent.
   * Reward: forward displacement of center-of-mass.

**Milestone:**

* Emergent locomotion in simulation.
* Visual verification via trails and node color.

---

## **Step 4: Morphology Evolution**

* Extend evolution to **graph structure itself** (add/remove nodes, edges).
* Fitness can encourage more efficient or adaptive body shapes.
* Explore emergent “gaits” like walking, rolling, or hopping.

**Milestone:** Automated growth + control: simulation discovers functional shapes and gaits.

---

## **Step 5: Analysis and Application**

1. **Analyze emergent gaits**:

   * Are there modular patterns, symmetry, or periodic contractions?
   * Can the model generalize to different terrains (friction, obstacles)?

2. **Potential outputs / publications**:

   * NCA for **graph-based locomotion**.
   * Evolution of morphology + local controllers.
   * Insights for **soft robots or modular robot design**.

---

## **Step 6: Optional Real-World Transfer**

* Convert the graph + contraction signals to **modular robot hardware** or **soft robotics simulation** (MuJoCo, PyBullet).
* Evaluate if locally-controlled “virtual creatures” can inspire **real soft-robot gaits**.

---

### 🔹 Suggested Milestone Timeline

| Step | Timeframe | Deliverable                            |
| ---- | --------- | -------------------------------------- |
| 0    | 1-2 weeks | Stable JS/Python playground            |
| 1    | 1-2 weeks | Fitness metrics implemented            |
| 2    | 3-4 weeks | First NCA/local-rule locomotion        |
| 3    | 4-6 weeks | Evolutionary locomotion emerges        |
| 4    | 6-8 weeks | Morphology + control co-evolution      |
| 5    | 2-4 weeks | Analysis, visualization, paper outline |
| 6    | Optional  | Simulation-to-hardware mapping         |

---

**Design a graph NCA controller model**, with inputs, outputs, and a small neural network per node that can be trained to contract edges locally.

